### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# sample num
sending_sample_num = 400  # 150,200,300,500

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = French
Lang Id = 3
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/3-Word Group In Youtube Sentence Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
##cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
#app = firebase_admin.initialize_app(cred)

In [7]:
#fire_db = firestore.client()

#### Read Data

In [8]:
#parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Result/2-Word Group Prepare For Youtube Tool"

In [9]:
#df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28 = df_28.head(sending_sample_num)
df_28

,video_id,start_time,end_time,text
0,YuxMdNbmxzM,1180,1182,mais cest pas pour ça que ça
1,aDBLqp0bIPw,672,673,cest pour ça que je vous le
2,GWSsZ6KCuJI,3402,3404,et cest pour ça que dans ce
3,KAgyTxbTNxw,926,929,cest pour ça que jai pas de
4,gpF9Kb4EC4M,1643,1646,cest pour ça que nous on en
...,...,...,...,...
395,LTRdb7Tvkbg,47,49,qui ça
396,RFRjqItUkqg,246,251,jai le
397,rAUkhIdYV_A,94,98,un des
398,s2l896FEzKc,315,317,on en


In [10]:
#df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200 = df_200.head(sending_sample_num)
df_200

,video_id,start_time,end_time,text
0,8ywehhHHhrU,618,624,dans la vie quand tout va bien mais aussi quan...
1,6NIUzeuw6Lk,951,960,je lui dis que je vais bien que voilà cest la ...
2,4YJSOd91bz4,775,779,donc là ce avait un peu de temps je sais pas p...
3,0_sv895JPlc,157,163,quand même quoi donc parce que voilà cest fait...
4,qTvOYQ_HMCA,159,165,là tu ne fais pas pour toi tu le fais parce qu...
...,...,...,...,...
286,f1JUZAXUzK4,34,37,en fait ils sont là avant nous donc nous on est
287,l6jmQ5_qrfs,2135,2138,alors dans un monde où tout est toujours un peu
288,Y1G3Uq7ytaw,313,318,besoin ils sont toujours là si jamais pour nous
289,okzZSymbYyY,2314,2317,comment vous avez fait en fait pour faire cette


In [11]:
#df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000 = df_1000.head(sending_sample_num)
df_1000

,video_id,start_time,end_time,text
0,mEwv2MslK10,386,391,pour le coup si tu comprends pas que tu peux p...
1,Tndgvn3FoZU,5913,5921,voilà oui on pourrait passer encore beaucoup p...
2,v0oH5e92pO0,3757,3763,bon moi je travaille tous les jours aujourdhui...
3,KZPJU5ggkJk,732,738,cest que cest pas pour tout de suite parce quà...
4,lMUZkh7Cbwc,1397,1403,la question je lui dis mais dis moi tu pourrai...
...,...,...,...,...
283,vfVZmKuBai0,972,976,avait des gens qui étaient là mais ils ont fai...
284,w7nvUyZUzc0,387,392,ce qui avait pu lui arriver elle avait tout au...
285,uo73C2ck1aU,476,480,est plus là pourquoi parce quils sont sur un a...
286,DLBcQ9pbYx0,8,12,suis dit quil fallait absolument faire un poin...


In [12]:
#df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000 = df_5000.head(sending_sample_num)
df_5000

,video_id,start_time,end_time,text
0,oig_hpR47OM,314,321,ah aussi je voulais vous dire je vais faire un...
1,sH7T97l3Bvo,238,245,femme mais non cest pas grave écoute de un ell...
2,8Kn1XH8MHsQ,14,26,faire le point puisque nous sommes bientôt fin...
3,5YPMBr4Nngg,985,997,ça jai pas de questions làdessus en revanche c...
4,_OfVDjtC93Y,486,501,si cest dangereux ou si elle perd le contrôle ...
...,...,...,...,...
182,NqpTyx9uRlg,133,138,sans même avoir essayé parce quen réalité il f...
183,lYwbO55v7TU,495,501,cest tout simplement une petite chanson pour s...
184,JzYwARg0NAg,1052,1058,cest la principale raison pour laquelle les ru...
185,CN50AFaGVN8,431,437,qui lui aussi avait rentré tous les livres dan...


In [13]:
#df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000 = df_10000.head(sending_sample_num)
df_10000

,video_id,start_time,end_time,text
0,YypPgUlKJt4,452,467,passé des moments heureux dans cette maison av...
1,oig_hpR47OM,314,321,ah aussi je voulais vous dire je vais faire un...
2,h6VaYkFE4yw,720,733,un coeur très différent et on ne peut pas amen...
3,eXE0dzAXErg,345,354,même si le meilleur moyen de réduire nos déche...
4,6Oissu5t7uE,78,87,pas remarqué alors pour le coup cest un person...
...,...,...,...,...
207,CUPmQSuh7Uo,4166,4172,pour certains mais ça changera rien récemment ...
208,FnHvNYsuCLE,12,16,au départ jy prête pas vraiment attention jai ...
209,hRF_qePrIUk,1516,1523,il faudrait pas quand même quune femme non mar...
210,4r44LGOIB5Q,415,423,si tout va bien leslie pourra se mettre debout...


In [14]:
#df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000 = df_20000.head(sending_sample_num)
df_20000

,video_id,start_time,end_time,text
0,eXE0dzAXErg,345,354,même si le meilleur moyen de réduire nos déche...
1,8Kn1XH8MHsQ,14,26,faire le point puisque nous sommes bientôt fin...
2,Cr9pam0A_O0,326,337,javais envie de javais besoin dexprimer au tra...
3,JwbVn3tYo8s,1183,1191,et nous voilà enfin devant théo pour la finale...
4,YypPgUlKJt4,418,429,cest tout ce que valent ces souvenirs cette po...
...,...,...,...,...
201,GWcvS_OswzQ,270,277,on est venu en cuisine on va bientôt les faire...
202,Cs4IoieomUc,1468,1475,alors on se dit pourquoi pas nous ce serait vr...
203,yzkU1NB2Rx4,901,908,un diagnostic plus poussé sera nécessaire ne r...
204,NqpTyx9uRlg,133,139,ou ignorer sans même avoir essayé parce quen r...


In [15]:
#df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000 = df_40000.head(sending_sample_num)
df_40000

,video_id,start_time,end_time,text
0,fcdtNH07Ip0,353,371,ils ne respectent pas toujours la réglementati...
1,OGkVgSZI8f8,641,662,arriver mathieu ne néglige rien il vise lexcel...
2,e2ITAibdtWo,365,377,mais comme toujours ce matériau nest pas parfa...
3,2T4jG33szys,50,62,jai mis dautres choses en place de lhypnose de...
4,NRAMYjBJ8Q4,89,102,choses ce que vous voulez et moi ici jai aussi...
...,...,...,...,...
208,ON4k3iaTOk8,628,633,si cest elle ce soir vous vous dites donc quel...
209,77BMeRZ9tZA,156,161,christophe cest différent en fait par exemple ...
210,g8FyvTePDIk,754,762,en disant mais ce nest pas possible quil ait c...
211,TacMMr3bieg,194,201,on est sur une bonne grosse boulette effective...


##### Calculate Speech Time Mean

In [16]:
(df_28["end_time"] - df_28["start_time"]).mean()

2.3575

In [17]:
(df_200["end_time"] - df_200["start_time"]).mean()

4.635738831615121

In [18]:
(df_1000["end_time"] - df_1000["start_time"]).mean()

6.072916666666667

In [19]:
(df_5000["end_time"] - df_5000["start_time"]).mean()

7.021390374331551

In [20]:
(df_10000["end_time"] - df_10000["start_time"]).mean()

7.589622641509434

In [21]:
(df_20000["end_time"] - df_20000["start_time"]).mean()

7.825242718446602

In [22]:
(df_40000["end_time"] - df_40000["start_time"]).mean()

8.2018779342723

#### Sending Data To Firestore

In [23]:
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [24]:
fire_db = firestore.client()

##### Language Data

In [25]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [26]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [27]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [28]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [29]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [30]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [31]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [32]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [33]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [34]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [35]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [36]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [37]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [38]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [39]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [40]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [41]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [42]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [43]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [44]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [45]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [46]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [47]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [48]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [49]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [50]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [51]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [52]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [53]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [54]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [55]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [56]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [57]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1684521421
  nanos: 108930000
}